# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


# Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove colunas constantes
colsToRemove = []
for col in df.columns:
    if df[col].std() == 0:
        colsToRemove.append(col)
df.drop(colsToRemove, axis=1, inplace=True)

## Remove colunas multiplas
colsToRemove = []
columns = df.columns
for i in range(len(columns)-1):
    print(i, end="\r")
    v = df[columns[i]]
    for j in range(i+1,len(columns)):
        m = df.loc[(df[columns[j]]*v) != 0, [columns[j], columns[i]]]
        if m.shape[0] > 0:
            c = m.iloc[0,0]/m.iloc[0,1]
            if np.array_equal(c*v.values,df[columns[j]].values):
                colsToRemove.append(columns[j])
df.drop(colsToRemove, axis=1, inplace=True)

## Remove duplicate rows
df.drop_duplicates(inplace=True)

## Split (X,Y) 
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 287)

# Oversampling: Random Oversampling

## 2 CNN 1 MAX RO

In [3]:
# info
model_name = '2 CNN 1 MAX RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97504 samples, validate on 13684 samples
Epoch 1/10000
97504/97504 [==============================] - 24s - loss: 0.5857 - acc: 0.6991 - val_loss: 0.5289 - val_acc: 0.7452
Epoch 2/10000
97504/97504 [==============================] - 14s - loss: 0.5121 - acc: 0.7499 - val_loss: 0.5461 - val_acc: 0.7212
Epoch 3/10000
97504/97504 [==============================] - 14s - loss: 0.4841 - acc: 0.7665 - val_loss: 0.4701 - val_acc: 0.7651
Epoch 4/10000
97504/97504 [==============================] - 14s - loss: 0.4649 - acc: 0.7780 - val_loss: 0.4828 - val_acc: 0.7363
Epoch 5/10000
97504/97504 [==============================] - 14s - loss: 0.4496 - acc: 0.7886 - val_loss: 0.4643 - val_acc: 0.7789
Epoch 6/10000
97504/97504 [==============================] - 14s - loss: 0.4320 - acc: 0.7987 - val_loss: 0.4504 - val_acc: 0.7963
Epoch 7/10000
97504/97504 [==============================] - 14s - loss: 0.4223 - acc: 0.8031 - val_loss: 0.4379 - val_acc: 0.7922
Epoch 8/10000
97504/97504 [=

97480/97480 [==============================] - 15s - loss: 0.4227 - acc: 0.7998 - val_loss: 0.4545 - val_acc: 0.7834
Epoch 10/10000
97480/97480 [==============================] - 14s - loss: 0.4114 - acc: 0.8062 - val_loss: 0.4223 - val_acc: 0.8091
Epoch 11/10000
97480/97480 [==============================] - 14s - loss: 0.4021 - acc: 0.8103 - val_loss: 0.4194 - val_acc: 0.7916
Epoch 12/10000
97480/97480 [==============================] - 14s - loss: 0.3935 - acc: 0.8165 - val_loss: 0.4439 - val_acc: 0.8056
Epoch 13/10000
97480/97480 [==============================] - 14s - loss: 0.3942 - acc: 0.8170 - val_loss: 0.4765 - val_acc: 0.7928
Epoch 14/10000
97480/97480 [==============================] - 14s - loss: 0.3969 - acc: 0.8159 - val_loss: 0.4765 - val_acc: 0.7786
Epoch 15/10000
97480/97480 [==============================] - 14s - loss: 0.3820 - acc: 0.8212 - val_loss: 0.4535 - val_acc: 0.8004
Epoch 16/10000
97480/97480 [==============================] - 14s - loss: 0.3749 - acc: 0.8

97486/97486 [==============================] - 14s - loss: 0.4062 - acc: 0.8096 - val_loss: 0.4736 - val_acc: 0.7867
Epoch 11/10000
97486/97486 [==============================] - 14s - loss: 0.3955 - acc: 0.8149 - val_loss: 0.4227 - val_acc: 0.8120
Epoch 12/10000
97486/97486 [==============================] - 14s - loss: 0.3870 - acc: 0.8216 - val_loss: 0.4620 - val_acc: 0.7990
Epoch 13/10000
97486/97486 [==============================] - 14s - loss: 0.3811 - acc: 0.8235 - val_loss: 0.4884 - val_acc: 0.7888
Epoch 14/10000
97486/97486 [==============================] - 14s - loss: 0.3764 - acc: 0.8279 - val_loss: 0.4560 - val_acc: 0.8050
Epoch 15/10000
97486/97486 [==============================] - 14s - loss: 0.3699 - acc: 0.8296 - val_loss: 0.4447 - val_acc: 0.8243
Epoch 16/10000
97486/97486 [==============================] - 14s - loss: 0.3635 - acc: 0.8325 - val_loss: 0.4835 - val_acc: 0.8018
Epoch 17/10000
97486/97486 [==============================] - 14s - loss: 0.3605 - acc: 0.8

97298/97298 [==============================] - 14s - loss: 0.3967 - acc: 0.8158 - val_loss: 0.4963 - val_acc: 0.7653
Epoch 11/10000
97298/97298 [==============================] - 14s - loss: 0.3867 - acc: 0.8219 - val_loss: 0.4093 - val_acc: 0.8280
Epoch 12/10000
97298/97298 [==============================] - 14s - loss: 0.3800 - acc: 0.8257 - val_loss: 0.5431 - val_acc: 0.7477
Epoch 13/10000
97298/97298 [==============================] - 14s - loss: 0.3719 - acc: 0.8296 - val_loss: 0.4737 - val_acc: 0.7933
Epoch 14/10000
97298/97298 [==============================] - 14s - loss: 0.3638 - acc: 0.8334 - val_loss: 0.5045 - val_acc: 0.7661
Epoch 15/10000
97298/97298 [==============================] - 14s - loss: 0.3634 - acc: 0.8352 - val_loss: 0.4642 - val_acc: 0.7876
Epoch 16/10000
97298/97298 [==============================] - 14s - loss: 0.3581 - acc: 0.8378 - val_loss: 0.5099 - val_acc: 0.7717
Epoch 17/10000
97298/97298 [==============================] - 14s - loss: 0.3516 - acc: 0.8

97504/97504 [==============================] - 14s - loss: 0.2972 - acc: 0.8728 - val_loss: 0.4899 - val_acc: 0.8207
Epoch 44/10000
97504/97504 [==============================] - 15s - loss: 0.2983 - acc: 0.8722 - val_loss: 0.4526 - val_acc: 0.8412
Epoch 45/10000
97504/97504 [==============================] - 14s - loss: 0.2928 - acc: 0.8724 - val_loss: 0.5048 - val_acc: 0.8059
Epoch 46/10000
97504/97504 [==============================] - 14s - loss: 0.2989 - acc: 0.8723 - val_loss: 0.4712 - val_acc: 0.8358
Epoch 47/10000
97504/97504 [==============================] - 14s - loss: 0.2935 - acc: 0.8751 - val_loss: 0.4586 - val_acc: 0.8310
Epoch 48/10000
97504/97504 [==============================] - 14s - loss: 0.3004 - acc: 0.8718 - val_loss: 0.4818 - val_acc: 0.8229
Epoch 49/10000
97504/97504 [==============================] - 14s - loss: 0.2939 - acc: 0.8733 - val_loss: 0.5170 - val_acc: 0.8138
Epoch 50/10000
97280/97504 [============================>.] - ETA: 0sKF_8
Train on 97518 sa

97416/97416 [==============================] - 14s - loss: 0.5581 - acc: 0.7162 - val_loss: 0.4946 - val_acc: 0.7498
Epoch 2/10000
97416/97416 [==============================] - 14s - loss: 0.5050 - acc: 0.7505 - val_loss: 0.5503 - val_acc: 0.7126
Epoch 3/10000
97416/97416 [==============================] - 14s - loss: 0.4875 - acc: 0.7615 - val_loss: 0.5311 - val_acc: 0.7286
Epoch 4/10000
97416/97416 [==============================] - 14s - loss: 0.4712 - acc: 0.7706 - val_loss: 0.5333 - val_acc: 0.7172
Epoch 5/10000
97416/97416 [==============================] - 14s - loss: 0.4567 - acc: 0.7802 - val_loss: 0.5274 - val_acc: 0.7071
Epoch 6/10000
97416/97416 [==============================] - 14s - loss: 0.4448 - acc: 0.7894 - val_loss: 0.4479 - val_acc: 0.7727
Epoch 7/10000
97416/97416 [==============================] - 14s - loss: 0.4346 - acc: 0.7943 - val_loss: 0.4519 - val_acc: 0.7732
Epoch 8/10000
97416/97416 [==============================] - 14s - loss: 0.4207 - acc: 0.8006 - v

In [4]:
print(model_name)

metrics_kf

2 CNN 1 MAX RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.950988,0.701448,0.701448,0.550545,0.694468,0.551588,0.826121,00:12:51.409793
KF_2,0.932056,0.732711,0.732711,0.550668,0.701080,0.550077,0.820466,00:07:52.202823
KF_3,0.933871,0.743154,0.743154,0.559221,0.705620,0.569076,0.847540,00:06:47.510860
KF_4,0.939034,0.756579,0.756579,0.554184,0.720403,0.553922,0.817811,00:07:51.578275
KF_5,0.938508,0.727316,0.727316,0.549335,0.701103,0.546447,0.814391,00:06:53.291636
KF_6,0.941353,0.758696,0.758696,0.553423,0.717081,0.552835,0.817548,00:06:47.234031
KF_7,0.952559,0.742800,0.742800,0.560084,0.707761,0.570509,0.848592,00:12:19.586715
KF_8,0.940830,0.732602,0.732602,0.547606,0.699272,0.542088,0.807814,00:06:47.921285
KF_9,0.940376,0.740095,0.740095,0.554162,0.708070,0.557258,0.829101,00:06:19.658640
KF_10,0.941531,0.716854,0.716854,0.546049,0.700378,0.537114,0.798974,00:08:03.901519


## 2 CNN 2 MAX RO

In [5]:
# info
model_name = '2 CNN 2 MAX RO Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97470 samples, validate on 13684 samples
Epoch 1/10000
97470/97470 [==============================] - 15s - loss: 0.6310 - acc: 0.6620 - val_loss: 0.5965 - val_acc: 0.6831
Epoch 2/10000
97470/97470 [==============================] - 15s - loss: 0.5531 - acc: 0.7175 - val_loss: 0.5521 - val_acc: 0.7292
Epoch 3/10000
97470/97470 [==============================] - 15s - loss: 0.5325 - acc: 0.7334 - val_loss: 0.5472 - val_acc: 0.6915
Epoch 4/10000
97470/97470 [==============================] - 15s - loss: 0.5261 - acc: 0.7363 - val_loss: 0.4967 - val_acc: 0.7710
Epoch 5/10000
97470/97470 [==============================] - 15s - loss: 0.5215 - acc: 0.7399 - val_loss: 0.5375 - val_acc: 0.7527
Epoch 6/10000
97470/97470 [==============================] - 15s - loss: 0.5175 - acc: 0.7426 - val_loss: 0.5251 - val_acc: 0.7651
Epoch 7/10000
97470/97470 [==============================] - 15s - loss: 0.5145 - acc: 0.7451 - val_loss: 0.5242 - val_acc: 0.7356
Epoch 8/10000
97470/97470 [=

97570/97570 [==============================] - 15s - loss: 0.5017 - acc: 0.7543 - val_loss: 0.5114 - val_acc: 0.7306
Epoch 18/10000
97570/97570 [==============================] - 15s - loss: 0.5005 - acc: 0.7557 - val_loss: 0.6243 - val_acc: 0.6535
Epoch 19/10000
97570/97570 [==============================] - 15s - loss: 0.4986 - acc: 0.7574 - val_loss: 0.5431 - val_acc: 0.7133
Epoch 20/10000
97570/97570 [==============================] - 15s - loss: 0.4974 - acc: 0.7573 - val_loss: 0.5769 - val_acc: 0.7152
Epoch 21/10000
97570/97570 [==============================] - 15s - loss: 0.4962 - acc: 0.7574 - val_loss: 0.5160 - val_acc: 0.7202
Epoch 22/10000
97570/97570 [==============================] - 15s - loss: 0.4957 - acc: 0.7590 - val_loss: 0.5637 - val_acc: 0.7069
Epoch 23/10000
97570/97570 [==============================] - 15s - loss: 0.4934 - acc: 0.7599 - val_loss: 0.5902 - val_acc: 0.6910
Epoch 24/10000
97570/97570 [==============================] - 15s - loss: 0.4918 - acc: 0.7

97580/97580 [==============================] - 15s - loss: 0.4739 - acc: 0.7692 - val_loss: 0.4961 - val_acc: 0.7496
Epoch 47/10000
97580/97580 [==============================] - 15s - loss: 0.4715 - acc: 0.7725 - val_loss: 0.4741 - val_acc: 0.7512
Epoch 48/10000
97580/97580 [==============================] - 15s - loss: 0.4713 - acc: 0.7715 - val_loss: 0.5565 - val_acc: 0.7174
Epoch 49/10000
97580/97580 [==============================] - 15s - loss: 0.4689 - acc: 0.7727 - val_loss: 0.5505 - val_acc: 0.7221
Epoch 50/10000
97580/97580 [==============================] - 15s - loss: 0.4674 - acc: 0.7725 - val_loss: 0.4834 - val_acc: 0.7549
Epoch 51/10000
97580/97580 [==============================] - 15s - loss: 0.4688 - acc: 0.7725 - val_loss: 0.5098 - val_acc: 0.7347
Epoch 52/10000
97580/97580 [==============================] - 15s - loss: 0.6109 - acc: 0.6955 - val_loss: 0.5843 - val_acc: 0.7353
Epoch 53/10000
97580/97580 [==============================] - 15s - loss: 0.5606 - acc: 0.7

97598/97598 [==============================] - 15s - loss: 0.4686 - acc: 0.7717 - val_loss: 0.5223 - val_acc: 0.7287
Epoch 54/10000
97598/97598 [==============================] - 15s - loss: 0.4653 - acc: 0.7736 - val_loss: 0.5513 - val_acc: 0.7219
Epoch 55/10000
97598/97598 [==============================] - 15s - loss: 0.4668 - acc: 0.7733 - val_loss: 0.5239 - val_acc: 0.7467
Epoch 56/10000
97598/97598 [==============================] - 15s - loss: 0.4639 - acc: 0.7740 - val_loss: 0.5660 - val_acc: 0.7394
Epoch 57/10000
97598/97598 [==============================] - 15s - loss: 0.4646 - acc: 0.7736 - val_loss: 0.5020 - val_acc: 0.7168
Epoch 58/10000
97504/97598 [============================>.] - ETA: 0sKF_5
Train on 97476 samples, validate on 13684 samples
Epoch 1/10000
97476/97476 [==============================] - 15s - loss: 0.6442 - acc: 0.6455 - val_loss: 0.6620 - val_acc: 0.7011
Epoch 2/10000
97476/97476 [==============================] - 15s - loss: 0.6116 - acc: 0.6839 - val_

97476/97476 [==============================] - 15s - loss: 0.4632 - acc: 0.7774 - val_loss: 0.5047 - val_acc: 0.7363
Epoch 57/10000
97476/97476 [==============================] - 15s - loss: 0.4584 - acc: 0.7790 - val_loss: 0.4652 - val_acc: 0.7645
Epoch 58/10000
97476/97476 [==============================] - 15s - loss: 0.4578 - acc: 0.7800 - val_loss: 0.5510 - val_acc: 0.7200
Epoch 59/10000
97476/97476 [==============================] - 15s - loss: 0.4597 - acc: 0.7775 - val_loss: 0.5739 - val_acc: 0.6512
Epoch 60/10000
97476/97476 [==============================] - 15s - loss: 0.4584 - acc: 0.7788 - val_loss: 0.5713 - val_acc: 0.7344
Epoch 61/10000
97476/97476 [==============================] - 15s - loss: 0.4651 - acc: 0.7774 - val_loss: 0.5183 - val_acc: 0.7342
Epoch 62/10000
97476/97476 [==============================] - 15s - loss: 0.4568 - acc: 0.7800 - val_loss: 0.5574 - val_acc: 0.7266
Epoch 63/10000
97476/97476 [==============================] - 15s - loss: 0.4554 - acc: 0.7

97414/97414 [==============================] - 15s - loss: 0.4910 - acc: 0.7553 - val_loss: 0.4735 - val_acc: 0.7675
Epoch 24/10000
97414/97414 [==============================] - 15s - loss: 0.4895 - acc: 0.7546 - val_loss: 0.5160 - val_acc: 0.7173
Epoch 25/10000
97056/97414 [============================>.] - ETA: 0sKF_7
Train on 97638 samples, validate on 13684 samples
Epoch 1/10000
97638/97638 [==============================] - 15s - loss: 0.6167 - acc: 0.6739 - val_loss: 0.5375 - val_acc: 0.7273
Epoch 2/10000
97638/97638 [==============================] - 15s - loss: 0.5481 - acc: 0.7224 - val_loss: 0.5065 - val_acc: 0.7592
Epoch 3/10000
97638/97638 [==============================] - 15s - loss: 0.5346 - acc: 0.7323 - val_loss: 0.5228 - val_acc: 0.7045
Epoch 4/10000
97638/97638 [==============================] - 15s - loss: 0.5263 - acc: 0.7357 - val_loss: 0.5410 - val_acc: 0.6966
Epoch 5/10000
97638/97638 [==============================] - 15s - loss: 0.5202 - acc: 0.7408 - val_los

97744/97744 [==============================] - 15s - loss: 0.5018 - acc: 0.7528 - val_loss: 0.5451 - val_acc: 0.6860
Epoch 14/10000
97744/97744 [==============================] - 15s - loss: 0.5013 - acc: 0.7530 - val_loss: 0.5121 - val_acc: 0.7195
Epoch 15/10000
97744/97744 [==============================] - 15s - loss: 0.4988 - acc: 0.7526 - val_loss: 0.5553 - val_acc: 0.7251
Epoch 16/10000
97744/97744 [==============================] - 15s - loss: 0.4978 - acc: 0.7545 - val_loss: 0.5535 - val_acc: 0.6950
Epoch 17/10000
97744/97744 [==============================] - 15s - loss: 0.4958 - acc: 0.7568 - val_loss: 0.4846 - val_acc: 0.7383
Epoch 18/10000
97744/97744 [==============================] - 15s - loss: 0.4959 - acc: 0.7568 - val_loss: 0.5754 - val_acc: 0.7026
Epoch 19/10000
97744/97744 [==============================] - 15s - loss: 0.4970 - acc: 0.7569 - val_loss: 0.4635 - val_acc: 0.7770
Epoch 20/10000
97744/97744 [==============================] - 15s - loss: 0.4930 - acc: 0.7

97580/97580 [==============================] - 15s - loss: 0.4805 - acc: 0.7648 - val_loss: 0.4979 - val_acc: 0.7585
Epoch 23/10000
97580/97580 [==============================] - 15s - loss: 0.4779 - acc: 0.7659 - val_loss: 0.5055 - val_acc: 0.7230
Epoch 24/10000
97580/97580 [==============================] - 15s - loss: 0.4790 - acc: 0.7650 - val_loss: 0.5489 - val_acc: 0.6910
Epoch 25/10000
97580/97580 [==============================] - 15s - loss: 0.4777 - acc: 0.7655 - val_loss: 0.4830 - val_acc: 0.7611
Epoch 26/10000
97580/97580 [==============================] - 15s - loss: 0.4747 - acc: 0.7681 - val_loss: 0.5168 - val_acc: 0.7501
Epoch 27/10000
97580/97580 [==============================] - 15s - loss: 0.4757 - acc: 0.7678 - val_loss: 0.5420 - val_acc: 0.6988
Epoch 28/10000
97580/97580 [==============================] - 15s - loss: 0.4786 - acc: 0.7672 - val_loss: 0.5379 - val_acc: 0.7192
Epoch 29/10000
97580/97580 [==============================] - 15s - loss: 0.4730 - acc: 0.7

In [6]:
print(model_name)

metrics_kf

2 CNN 2 MAX RO Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.862854,0.793415,0.793415,0.542679,0.731188,0.506848,0.728397,00:11:21.120890
KF_2,0.848593,0.808455,0.811777,0.545759,0.740381,0.516297,0.742996,00:08:18.025745
KF_3,0.817241,0.801765,0.801765,0.538915,0.729566,0.483513,0.682452,00:13:55.532583
KF_4,0.862469,0.794669,0.794669,0.541825,0.727094,0.505155,0.726651,00:14:56.461828
KF_5,0.877938,0.767791,0.767791,0.544621,0.715905,0.524997,0.769403,00:24:14.220092
KF_6,0.836650,0.800289,0.800289,0.540712,0.733691,0.492742,0.699553,00:06:29.698283
KF_7,0.861583,0.806084,0.809406,0.543778,0.746499,0.501348,0.711918,00:11:35.954228
KF_8,0.858231,0.773250,0.773250,0.540644,0.722830,0.501720,0.721521,00:13:23.912562
KF_9,0.863708,0.787133,0.790467,0.542752,0.727324,0.510345,0.737140,00:09:02.047096
KF_10,0.833867,0.801576,0.801576,0.544473,0.737963,0.511958,0.736087,00:07:14.001225


# Oversampling: SMOTE

## 2 CNN 1 MAX SM

In [7]:
# info
model_name = '2 CNN 1 MAX SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97567 samples, validate on 13684 samples
Epoch 1/10000
97567/97567 [==============================] - 14s - loss: 0.4554 - acc: 0.7973 - val_loss: 0.3005 - val_acc: 0.8765
Epoch 2/10000
97567/97567 [==============================] - 14s - loss: 0.3348 - acc: 0.8642 - val_loss: 0.3384 - val_acc: 0.8527
Epoch 3/10000
97567/97567 [==============================] - 14s - loss: 0.3088 - acc: 0.8751 - val_loss: 0.3688 - val_acc: 0.8352
Epoch 4/10000
97567/97567 [==============================] - 14s - loss: 0.2937 - acc: 0.8824 - val_loss: 0.2757 - val_acc: 0.8780
Epoch 5/10000
97567/97567 [==============================] - 14s - loss: 0.2830 - acc: 0.8887 - val_loss: 0.3205 - val_acc: 0.8559
Epoch 6/10000
97567/97567 [==============================] - 14s - loss: 0.2750 - acc: 0.8917 - val_loss: 0.3868 - val_acc: 0.8208
Epoch 7/10000
97567/97567 [==============================] - 14s - loss: 0.2709 - acc: 0.8941 - val_loss: 0.3972 - val_acc: 0.8356
Epoch 8/10000
97567/97567 [=

97669/97669 [==============================] - 14s - loss: 0.4555 - acc: 0.8005 - val_loss: 0.3158 - val_acc: 0.8409
Epoch 2/10000
97669/97669 [==============================] - 14s - loss: 0.3429 - acc: 0.8574 - val_loss: 0.2416 - val_acc: 0.8854
Epoch 3/10000
97669/97669 [==============================] - 14s - loss: 0.3204 - acc: 0.8686 - val_loss: 0.1841 - val_acc: 0.9328
Epoch 4/10000
97669/97669 [==============================] - 14s - loss: 0.3060 - acc: 0.8747 - val_loss: 0.2379 - val_acc: 0.8843
Epoch 5/10000
97669/97669 [==============================] - 14s - loss: 0.2939 - acc: 0.8813 - val_loss: 0.2542 - val_acc: 0.8846
Epoch 6/10000
97669/97669 [==============================] - 14s - loss: 0.2861 - acc: 0.8843 - val_loss: 0.2203 - val_acc: 0.8969
Epoch 7/10000
97669/97669 [==============================] - 14s - loss: 0.2793 - acc: 0.8872 - val_loss: 1.2673 - val_acc: 0.5210
Epoch 8/10000
97669/97669 [==============================] - 14s - loss: 0.2845 - acc: 0.8849 - v

97616/97616 [==============================] - 14s - loss: 0.2132 - acc: 0.9170 - val_loss: 0.3744 - val_acc: 0.8584
Epoch 39/10000
97616/97616 [==============================] - 14s - loss: 0.2105 - acc: 0.9188 - val_loss: 0.3515 - val_acc: 0.8673
Epoch 40/10000
97616/97616 [==============================] - 14s - loss: 0.2108 - acc: 0.9182 - val_loss: 0.3464 - val_acc: 0.8699
Epoch 41/10000
97616/97616 [==============================] - 14s - loss: 0.2133 - acc: 0.9175 - val_loss: 0.3212 - val_acc: 0.8769
Epoch 42/10000
97616/97616 [==============================] - 14s - loss: 0.2114 - acc: 0.9187 - val_loss: 0.3469 - val_acc: 0.8809
Epoch 43/10000
97616/97616 [==============================] - 14s - loss: 0.2077 - acc: 0.9192 - val_loss: 0.3614 - val_acc: 0.8782
Epoch 44/10000
97616/97616 [==============================] - 14s - loss: 0.2081 - acc: 0.9193 - val_loss: 0.3173 - val_acc: 0.8823
Epoch 45/10000
97616/97616 [==============================] - 14s - loss: 0.2059 - acc: 0.9

97462/97462 [==============================] - 14s - loss: 0.2238 - acc: 0.9157 - val_loss: 0.2903 - val_acc: 0.8794
Epoch 26/10000
97462/97462 [==============================] - 14s - loss: 0.2222 - acc: 0.9160 - val_loss: 0.3375 - val_acc: 0.8661
Epoch 27/10000
97462/97462 [==============================] - 14s - loss: 0.2238 - acc: 0.9153 - val_loss: 0.3498 - val_acc: 0.8618
Epoch 28/10000
97462/97462 [==============================] - 14s - loss: 0.2213 - acc: 0.9159 - val_loss: 0.3066 - val_acc: 0.8722
Epoch 29/10000
97462/97462 [==============================] - 14s - loss: 0.2206 - acc: 0.9163 - val_loss: 0.3096 - val_acc: 0.8733
Epoch 30/10000
97462/97462 [==============================] - 14s - loss: 0.2168 - acc: 0.9182 - val_loss: 0.3756 - val_acc: 0.8605
Epoch 31/10000
97462/97462 [==============================] - 14s - loss: 0.2186 - acc: 0.9165 - val_loss: 0.3017 - val_acc: 0.8804
Epoch 32/10000
97462/97462 [==============================] - 14s - loss: 0.2172 - acc: 0.9

97623/97623 [==============================] - 14s - loss: 0.2343 - acc: 0.9066 - val_loss: 0.3675 - val_acc: 0.8527
Epoch 23/10000
97623/97623 [==============================] - 14s - loss: 0.2341 - acc: 0.9077 - val_loss: 0.4254 - val_acc: 0.8233
Epoch 24/10000
97623/97623 [==============================] - 14s - loss: 0.2315 - acc: 0.9086 - val_loss: 0.3324 - val_acc: 0.8580
Epoch 25/10000
97623/97623 [==============================] - 14s - loss: 0.2319 - acc: 0.9083 - val_loss: 0.2817 - val_acc: 0.8756
Epoch 26/10000
97623/97623 [==============================] - 14s - loss: 0.2281 - acc: 0.9093 - val_loss: 0.3421 - val_acc: 0.8636
Epoch 27/10000
97623/97623 [==============================] - 14s - loss: 0.2267 - acc: 0.9102 - val_loss: 0.3221 - val_acc: 0.8761
Epoch 28/10000
97623/97623 [==============================] - 14s - loss: 0.2264 - acc: 0.9101 - val_loss: 0.2918 - val_acc: 0.8829
Epoch 29/10000
97623/97623 [==============================] - 14s - loss: 0.2261 - acc: 0.9

97505/97505 [==============================] - 14s - loss: 0.4195 - acc: 0.8179 - val_loss: 0.3668 - val_acc: 0.8348
Epoch 2/10000
97505/97505 [==============================] - 14s - loss: 0.3369 - acc: 0.8604 - val_loss: 0.4404 - val_acc: 0.7869
Epoch 3/10000
97505/97505 [==============================] - 14s - loss: 0.3171 - acc: 0.8696 - val_loss: 0.3460 - val_acc: 0.8462
Epoch 4/10000
97505/97505 [==============================] - 14s - loss: 0.3035 - acc: 0.8768 - val_loss: 0.3414 - val_acc: 0.8454
Epoch 5/10000
97505/97505 [==============================] - 14s - loss: 0.2922 - acc: 0.8819 - val_loss: 0.3308 - val_acc: 0.8555
Epoch 6/10000
97505/97505 [==============================] - 14s - loss: 0.2855 - acc: 0.8857 - val_loss: 0.3597 - val_acc: 0.8412
Epoch 7/10000
97505/97505 [==============================] - 14s - loss: 0.2787 - acc: 0.8879 - val_loss: 0.3354 - val_acc: 0.8548
Epoch 8/10000
97505/97505 [==============================] - 14s - loss: 0.2744 - acc: 0.8900 - v

In [8]:
print(model_name)

metrics_kf

2 CNN 1 MAX SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.971992,0.758457,0.758457,0.563016,0.670962,0.579847,0.878864,00:07:49.428818
KF_2,0.966900,0.764628,0.767950,0.571775,0.717818,0.591122,0.870972,00:06:40.334425
KF_3,0.964258,0.744983,0.744983,0.568109,0.694793,0.586633,0.875691,00:05:51.506855
KF_4,0.973859,0.762430,0.765752,0.570932,0.698235,0.590947,0.879242,00:11:17.208461
KF_5,0.968572,0.773575,0.773575,0.559139,0.698530,0.570076,0.852276,00:06:38.880957
KF_6,0.971870,0.779855,0.779855,0.573353,0.715292,0.593916,0.875296,00:09:04.747967
KF_7,0.968661,0.777625,0.780947,0.568888,0.700411,0.587546,0.874244,00:06:24.378922
KF_8,0.973546,0.718526,0.718526,0.563538,0.671366,0.580642,0.879637,00:11:15.730101
KF_9,0.969905,0.764987,0.764987,0.563085,0.711836,0.576029,0.854756,00:08:36.806402
KF_10,0.968605,0.759575,0.759575,0.564473,0.697428,0.580344,0.866991,00:07:35.477987


## 2 CNN 2 MAX SM

In [9]:
# info
model_name = '2 CNN 2 MAX SM Clean'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,2),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 17, 17, 1)
    x_test = x_test.reshape(x_test.shape[0], 17, 17, 1)
    x_val = x_val.reshape(x_val.shape[0], 17, 17, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(17, 17, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97520 samples, validate on 13684 samples
Epoch 1/10000
97520/97520 [==============================] - 15s - loss: 0.5420 - acc: 0.7579 - val_loss: 0.5040 - val_acc: 0.7005
Epoch 2/10000
97520/97520 [==============================] - 15s - loss: 0.4310 - acc: 0.8051 - val_loss: 0.3892 - val_acc: 0.8295
Epoch 3/10000
97520/97520 [==============================] - 15s - loss: 0.3688 - acc: 0.8462 - val_loss: 0.4578 - val_acc: 0.7931
Epoch 4/10000
97520/97520 [==============================] - 15s - loss: 0.3487 - acc: 0.8549 - val_loss: 0.3430 - val_acc: 0.8399
Epoch 5/10000
97520/97520 [==============================] - 15s - loss: 0.3423 - acc: 0.8587 - val_loss: 0.3429 - val_acc: 0.8601
Epoch 6/10000
97520/97520 [==============================] - 15s - loss: 0.3381 - acc: 0.8607 - val_loss: 0.4052 - val_acc: 0.8305
Epoch 7/10000
97520/97520 [==============================] - 15s - loss: 0.3292 - acc: 0.8647 - val_loss: 0.3022 - val_acc: 0.8628
Epoch 8/10000
97520/97520 [=

97455/97455 [==============================] - 15s - loss: 0.3176 - acc: 0.8699 - val_loss: 0.3163 - val_acc: 0.8503
Epoch 14/10000
97455/97455 [==============================] - 15s - loss: 0.3140 - acc: 0.8707 - val_loss: 0.3582 - val_acc: 0.8425
Epoch 15/10000
97455/97455 [==============================] - 15s - loss: 0.3140 - acc: 0.8712 - val_loss: 0.3356 - val_acc: 0.8563
Epoch 16/10000
97455/97455 [==============================] - 15s - loss: 0.3121 - acc: 0.8723 - val_loss: 0.3083 - val_acc: 0.8577
Epoch 17/10000
97455/97455 [==============================] - 15s - loss: 0.3117 - acc: 0.8728 - val_loss: 0.2794 - val_acc: 0.8693
Epoch 18/10000
97455/97455 [==============================] - 15s - loss: 0.3105 - acc: 0.8728 - val_loss: 0.3621 - val_acc: 0.8482
Epoch 19/10000
97455/97455 [==============================] - 15s - loss: 0.3099 - acc: 0.8732 - val_loss: 0.3746 - val_acc: 0.8411
Epoch 20/10000
97455/97455 [==============================] - 15s - loss: 0.3114 - acc: 0.8

97595/97595 [==============================] - 15s - loss: 0.2968 - acc: 0.8785 - val_loss: 0.3730 - val_acc: 0.8354
Epoch 48/10000
97595/97595 [==============================] - 15s - loss: 0.2967 - acc: 0.8786 - val_loss: 0.3595 - val_acc: 0.8403
Epoch 49/10000
97595/97595 [==============================] - 15s - loss: 0.3042 - acc: 0.8743 - val_loss: 0.4706 - val_acc: 0.8089
Epoch 50/10000
97595/97595 [==============================] - 15s - loss: 0.2954 - acc: 0.8789 - val_loss: 0.3356 - val_acc: 0.8516
Epoch 51/10000
97595/97595 [==============================] - 15s - loss: 0.2977 - acc: 0.8781 - val_loss: 0.2575 - val_acc: 0.8788
Epoch 52/10000
97595/97595 [==============================] - 15s - loss: 0.2947 - acc: 0.8793 - val_loss: 0.5050 - val_acc: 0.7889
Epoch 53/10000
97595/97595 [==============================] - 15s - loss: 0.2947 - acc: 0.8800 - val_loss: 0.4313 - val_acc: 0.8118
Epoch 54/10000
97595/97595 [==============================] - 15s - loss: 0.2977 - acc: 0.8

97499/97499 [==============================] - 15s - loss: 0.3046 - acc: 0.8730 - val_loss: 0.3641 - val_acc: 0.8354
Epoch 37/10000
97499/97499 [==============================] - 15s - loss: 0.3030 - acc: 0.8741 - val_loss: 0.3203 - val_acc: 0.8558
Epoch 38/10000
97499/97499 [==============================] - 15s - loss: 0.3028 - acc: 0.8742 - val_loss: 0.4188 - val_acc: 0.8060
Epoch 39/10000
97499/97499 [==============================] - 15s - loss: 0.3030 - acc: 0.8750 - val_loss: 0.4025 - val_acc: 0.8169
Epoch 40/10000
97499/97499 [==============================] - 15s - loss: 0.3008 - acc: 0.8748 - val_loss: 0.3193 - val_acc: 0.8485
Epoch 41/10000
97056/97499 [============================>.] - ETA: 0sKF_5
Train on 97400 samples, validate on 13684 samples
Epoch 1/10000
97400/97400 [==============================] - 15s - loss: 0.5576 - acc: 0.7442 - val_loss: 0.4702 - val_acc: 0.7371
Epoch 2/10000
97400/97400 [==============================] - 15s - loss: 0.4636 - acc: 0.7940 - val_

97484/97484 [==============================] - 15s - loss: 0.3287 - acc: 0.8638 - val_loss: 0.3776 - val_acc: 0.8290
Epoch 12/10000
97484/97484 [==============================] - 15s - loss: 0.3268 - acc: 0.8637 - val_loss: 0.3103 - val_acc: 0.8435
Epoch 13/10000
97484/97484 [==============================] - 15s - loss: 0.3251 - acc: 0.8660 - val_loss: 0.3098 - val_acc: 0.8559
Epoch 14/10000
97484/97484 [==============================] - 15s - loss: 0.3226 - acc: 0.8670 - val_loss: 0.3090 - val_acc: 0.8627
Epoch 15/10000
97484/97484 [==============================] - 15s - loss: 0.3215 - acc: 0.8680 - val_loss: 0.3604 - val_acc: 0.8460
Epoch 16/10000
97484/97484 [==============================] - 15s - loss: 0.3217 - acc: 0.8665 - val_loss: 0.3401 - val_acc: 0.8456
Epoch 17/10000
97484/97484 [==============================] - 15s - loss: 0.3178 - acc: 0.8685 - val_loss: 0.3587 - val_acc: 0.8441
Epoch 18/10000
97484/97484 [==============================] - 15s - loss: 0.3174 - acc: 0.8

97521/97521 [==============================] - 15s - loss: 0.3047 - acc: 0.8757 - val_loss: 0.3427 - val_acc: 0.8447
Epoch 29/10000
97521/97521 [==============================] - 15s - loss: 0.3050 - acc: 0.8751 - val_loss: 0.3552 - val_acc: 0.8447
Epoch 30/10000
97521/97521 [==============================] - 15s - loss: 0.3040 - acc: 0.8760 - val_loss: 0.3587 - val_acc: 0.8434
Epoch 31/10000
97521/97521 [==============================] - 15s - loss: 0.3020 - acc: 0.8768 - val_loss: 0.3626 - val_acc: 0.8468
Epoch 32/10000
97521/97521 [==============================] - 15s - loss: 0.3027 - acc: 0.8772 - val_loss: 0.3090 - val_acc: 0.8573
Epoch 33/10000
97521/97521 [==============================] - 15s - loss: 0.3018 - acc: 0.8772 - val_loss: 0.2871 - val_acc: 0.8660
Epoch 34/10000
97521/97521 [==============================] - 15s - loss: 0.3035 - acc: 0.8767 - val_loss: 0.3497 - val_acc: 0.8432
Epoch 35/10000
97521/97521 [==============================] - 15s - loss: 0.3008 - acc: 0.8

97611/97611 [==============================] - 15s - loss: 0.3240 - acc: 0.8653 - val_loss: 0.3713 - val_acc: 0.8272
Epoch 11/10000
97611/97611 [==============================] - 15s - loss: 0.3210 - acc: 0.8658 - val_loss: 0.3446 - val_acc: 0.8394
Epoch 12/10000
97611/97611 [==============================] - 15s - loss: 0.3211 - acc: 0.8671 - val_loss: 0.3674 - val_acc: 0.8294
Epoch 13/10000
97611/97611 [==============================] - 15s - loss: 0.3140 - acc: 0.8700 - val_loss: 0.3380 - val_acc: 0.8451
Epoch 14/10000
97611/97611 [==============================] - 15s - loss: 0.3174 - acc: 0.8694 - val_loss: 0.3480 - val_acc: 0.8443
Epoch 15/10000
97611/97611 [==============================] - 15s - loss: 0.3142 - acc: 0.8698 - val_loss: 0.3290 - val_acc: 0.8565
Epoch 16/10000
97611/97611 [==============================] - 15s - loss: 0.3115 - acc: 0.8695 - val_loss: 0.3543 - val_acc: 0.8383
Epoch 17/10000
97611/97611 [==============================] - 15s - loss: 0.3098 - acc: 0.8

97454/97454 [==============================] - 15s - loss: 0.3217 - acc: 0.8659 - val_loss: 0.3646 - val_acc: 0.8384
Epoch 34/10000
97454/97454 [==============================] - 15s - loss: 0.3212 - acc: 0.8659 - val_loss: 0.3723 - val_acc: 0.8298
Epoch 35/10000
97216/97454 [============================>.] - ETA: 0s


In [10]:
print(model_name)

metrics_kf

2 CNN 2 MAX SM Clean


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.953077,0.797298,0.797298,0.558856,0.700560,0.569177,0.850059,00:12:21.635860
KF_2,0.944818,0.808749,0.808749,0.581316,0.723621,0.605985,0.885177,00:06:56.167042
KF_3,0.951240,0.796218,0.796218,0.556590,0.717216,0.560824,0.830045,00:18:35.562767
KF_4,0.945835,0.793432,0.793432,0.564147,0.702416,0.579153,0.862799,00:10:35.272420
KF_5,0.945706,0.767875,0.767875,0.559007,0.703051,0.569069,0.848724,00:11:21.743723
KF_6,0.947614,0.809041,0.809041,0.560381,0.730145,0.566892,0.833465,00:11:17.483953
KF_7,0.949473,0.797278,0.797278,0.559175,0.710210,0.568161,0.844120,00:11:38.913412
KF_8,0.948889,0.769020,0.769020,0.551754,0.681902,0.557471,0.841752,00:08:34.471285
KF_9,0.949080,0.780032,0.783365,0.561867,0.705785,0.574501,0.855414,00:09:45.990030
KF_10,0.941818,0.796934,0.796934,0.559629,0.709256,0.569410,0.846731,00:09:05.130895


# Metrics

In [11]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
2 CNN 1 MAX RO Clean,0.941110,0.735225,0.735225,0.552528,0.705524,0.553091,0.822836,00:08:15.429557
2 CNN 2 MAX RO Clean,0.852313,0.793443,0.794440,0.542616,0.731244,0.505492,0.725612,00:12:03.097453
2 CNN 1 MAX SM Clean,0.969817,0.760464,0.761461,0.566631,0.697667,0.583710,0.870797,00:08:07.450089
2 CNN 2 MAX SM Clean,0.947755,0.791588,0.791921,0.561272,0.708416,0.572064,0.849829,00:11:01.237138
